In [ ]:
from google.cloud import bigquery
from dotenv import load_dotenv
import os
import pandera as pa

from sqlguard.validator.SQLValidator import SQLValidator
from sqlgguard.translators import SchemaParsers

import pprint
import json

import pandas as pd
import duckdb

load_dotenv()

ModuleNotFoundError: No module named 'sqlguard'

# Getting Data From BigQuery

It's a table thar contains information about students and their courses and grades. Imagine those students are from a Computer Science Major or from a Electrical Engineering Major.  
The ideia is to have each row as the grade of a student.  
```
name: Name of the student  
age: Age of the student  
major: Major of the student (for example, Computer Science, Electrical Engineering etc.)  
semester: Semester of university, for example 1S/2024 indicating the first semester of 2024  
course: Course, for example, Algorithms, Data Structures, Calculus I, Calculus II etc.  
grade: Grade for the course. for example: 0, 5, 10  
failed: Boolean to indicate if the student failed the course based on grade (>=5)  
```

In [9]:
TABLE_PATH = "`central-rampart-451901-k9.test.student`"

In [10]:
client = bigquery.Client.from_service_account_json(json_credentials_path=os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))

# Perform a query.
QUERY = f'''

    SELECT * FROM {TABLE_PATH}

'''

query_job = client.query(QUERY)  # API request
query_result = query_job.result()  # Waits for query to finish

df = query_result.to_dataframe()

df

/home/ygorj/Code/sql-guard/env/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,name,age,major,semester,course,grade,failed
0,Daniel Carter,19,Computer Science,1S/2024,Algorithms,8.5,False
1,Theo Hill,19,Computer Science,1S/2024,Algorithms,9.0,False
2,Jessica Hall,19,Computer Science,1S/2024,Algorithms,8.0,False
3,Liam Carter,19,Computer Science,1S/2024,Algorithms,8.5,False
4,Zackary Hill,19,Computer Science,1S/2024,Algorithms,7.0,False
...,...,...,...,...,...,...,...
98,Nora Turner,21,Computer Science,1S/2024,Data Structures,6.0,True
99,Riley Turner,21,Computer Science,1S/2024,Data Structures,5.0,True
100,Leo Adams,21,Computer Science,1S/2024,Data Structures,5.5,True
101,Fiona Lopez,21,Computer Science,1S/2024,Data Structures,5.0,True


# Validating

## Setting up pa.DataFrameSchema

In [11]:
# Variables that we are going to use

pandera_schema = pa.DataFrameSchema({

    "name": pa.Column(str, checks=pa.Check.str_matches(r"^[A-Z].*")), # Starting with capital letter
    "age": pa.Column(int, checks=pa.Check.in_range(min_value=15, max_value=150)), # Students must be between 15 and 150 years old
    "major": pa.Column(str, checks=pa.Check.isin(["Computer Science", "Electrical Engineering"])), # Major can only be Computer Science or Electrical Engineering
    "semester": pa.Column(str, checks=pa.Check.equal_to("1S/2024")), # Only grade from 1st semester of 2024
    "course": pa.Column(str, checks=pa.Check.isin(["Algorithms", "Data Structures", "Circuit Analysis", "Calculus I", "Calculus II"])), # Only Alogirthms and Data Structures courses
    "grade": pa.Column(float, checks=pa.Check.between(min_value=0, max_value=10)), # Grade can only be between 0 and 10
    "failed": pa.Column(bool) # Boolean value
})

print(pandera_schema)

<Schema DataFrameSchema(
    columns={
        'name': <Schema Column(name=name, type=DataType(str))>
        'age': <Schema Column(name=age, type=DataType(int64))>
        'major': <Schema Column(name=major, type=DataType(str))>
        'semester': <Schema Column(name=semester, type=DataType(str))>
        'course': <Schema Column(name=course, type=DataType(str))>
        'grade': <Schema Column(name=grade, type=DataType(float64))>
        'failed': <Schema Column(name=failed, type=DataType(bool))>
    },
    checks=[],
    parsers=[],
    coerce=False,
    dtype=None,
    index=None,
    strict=False,
    name=None,
    ordered=False,
    unique_column_names=False,
    metadata=None, 
    add_missing_columns=False
)>


In [12]:
pandera_schema = pa.DataFrameSchema({

    "name": pa.Column(str, checks=pa.Check.str_matches(r"^[A-Z].*")), # Starting with capital letter
    "age": pa.Column(int, checks=pa.Check.in_range(min_value=15, max_value=21)), # Students must be between 15 and 150 years old
    "major": pa.Column(str, checks=pa.Check.isin(["Computer Science"])), # Major can only be Computer Science
    "semester": pa.Column(str, checks=pa.Check.equal_to("1S/2024")), # Only grade from 1st semester of 2024
    "course": pa.Column(str, checks=pa.Check.isin(["Algorithms", "Data Structures", "Calculus I"])), # Only Alogirthms and Data Structures courses
    "grade": pa.Column(float, checks=pa.Check.between(min_value=0, max_value=10)), # Grade can only be between 0 and 10
    "failed": pa.Column(bool) # Boolean value
})

print(pandera_schema)

<Schema DataFrameSchema(
    columns={
        'name': <Schema Column(name=name, type=DataType(str))>
        'age': <Schema Column(name=age, type=DataType(int64))>
        'major': <Schema Column(name=major, type=DataType(str))>
        'semester': <Schema Column(name=semester, type=DataType(str))>
        'course': <Schema Column(name=course, type=DataType(str))>
        'grade': <Schema Column(name=grade, type=DataType(float64))>
        'failed': <Schema Column(name=failed, type=DataType(bool))>
    },
    checks=[],
    parsers=[],
    coerce=False,
    dtype=None,
    index=None,
    strict=False,
    name=None,
    ordered=False,
    unique_column_names=False,
    metadata=None, 
    add_missing_columns=False
)>


## Validating with SQL Guard

In [13]:
panderaParser = SchemaParsers.SchemaParser.get_parser("pandera")
data_rules = panderaParser.parse(pandera_schema)
pprint.pp(data_rules)

{'name': [ValidationCheck(check_name='is_string',
                          params=None,
                          error_msg=None,
                          ignore_nulls=False),
          ValidationCheck(check_name='regex_contains',
                          params={'value': '^[A-Z].*'},
                          error_msg=None,
                          ignore_nulls=False)],
 'age': [ValidationCheck(check_name='is_integer',
                         params=None,
                         error_msg=None,
                         ignore_nulls=False),
         ValidationCheck(check_name='between',
                         params={'min': 15, 'max': 21},
                         error_msg=None,
                         ignore_nulls=False)],
 'major': [ValidationCheck(check_name='is_string',
                           params=None,
                           error_msg=None,
                           ignore_nulls=False),
           ValidationCheck(check_name='is_in',
                          

In [14]:
sql_schema = SQLValidator(data_rules, sql_dialect="DuckDBSQL")
validation_query = sql_schema.generate_sql(from_source="df")

print(validation_query)


SELECT * FROM df
            WHERE ((TRY_CAST(name AS VARCHAR) IS NOT NULL) AND (REGEXP_MATCHES(name, E'^[A-Z].*'))) AND ((TRY_CAST(age AS BIGINT) IS NOT NULL) AND (age BETWEEN 15 AND 21)) AND ((TRY_CAST(major AS VARCHAR) IS NOT NULL) AND (major IN ('Computer Science'))) AND ((TRY_CAST(semester AS VARCHAR) IS NOT NULL) AND (semester = '1S/2024')) AND ((TRY_CAST(course AS VARCHAR) IS NOT NULL) AND (course IN ('Algorithms', 'Data Structures', 'Calculus I'))) AND ((TRY_CAST(grade AS DOUBLE) IS NOT NULL) AND (grade BETWEEN 0 AND 10)) AND ((TRY_CAST(failed AS BOOLEAN) IS NOT NULL))


In [15]:
duckdb.sql(validation_query)

┌───────────────┬───────┬──────────────────┬──────────┬─────────────────┬────────┬─────────┐
│     name      │  age  │      major       │ semester │     course      │ grade  │ failed  │
│    varchar    │ int64 │     varchar      │ varchar  │     varchar     │ double │ boolean │
├───────────────┼───────┼──────────────────┼──────────┼─────────────────┼────────┼─────────┤
│ Daniel Carter │    19 │ Computer Science │ 1S/2024  │ Algorithms      │    8.5 │ false   │
│ Theo Hill     │    19 │ Computer Science │ 1S/2024  │ Algorithms      │    9.0 │ false   │
│ Jessica Hall  │    19 │ Computer Science │ 1S/2024  │ Algorithms      │    8.0 │ false   │
│ Liam Carter   │    19 │ Computer Science │ 1S/2024  │ Algorithms      │    8.5 │ false   │
│ Zackary Hill  │    19 │ Computer Science │ 1S/2024  │ Algorithms      │    7.0 │ false   │
│ Zoe Carter    │    19 │ Computer Science │ 1S/2024  │ Algorithms      │    9.0 │ false   │
│ John Doe      │    20 │ Computer Science │ 1S/2024  │ Algorithms    

In [16]:
validation_query = sql_schema.generate_sql(from_source="df", wrong_values=True)

duckdb.sql(validation_query)

┌───────────────┬───────┬────────────────────────┬──────────┬──────────────────┬────────┬─────────┐
│     name      │  age  │         major          │ semester │      course      │ grade  │ failed  │
│    varchar    │ int64 │        varchar         │ varchar  │     varchar      │ double │ boolean │
├───────────────┼───────┼────────────────────────┼──────────┼──────────────────┼────────┼─────────┤
│ Maya Baker    │    22 │ Electrical Engineering │ 1S/2024  │ Calculus I       │    8.5 │ false   │
│ Caleb White   │    22 │ Electrical Engineering │ 1S/2024  │ Calculus I       │    7.5 │ false   │
│ Grace White   │    22 │ Electrical Engineering │ 1S/2024  │ Calculus I       │    9.5 │ false   │
│ Yara Gonzalez │    22 │ Electrical Engineering │ 1S/2024  │ Calculus I       │    8.0 │ false   │
│ Olivia Baker  │    22 │ Electrical Engineering │ 1S/2024  │ Calculus I       │    7.5 │ false   │
│ Uma Gonzalez  │    22 │ Electrical Engineering │ 1S/2024  │ Calculus I       │    9.5 │ false   │


In [19]:
validation_query = sql_schema.generate_sql_report(from_source="df", n_wrong_values=True)

duckdb.sql(validation_query)

┌─────────────┬────────────┬────────────────────────────────────────────────────────────┬───────────┬──────────────┬────────────────┐
│ column_name │ check_name │                           params                           │ error_msg │ ignore_nulls │ n_wrong_values │
│   varchar   │  varchar   │                          varchar                           │   int32   │   boolean    │     int64      │
├─────────────┼────────────┼────────────────────────────────────────────────────────────┼───────────┼──────────────┼────────────────┤
│ major       │ is_in      │ {"value": ["Computer Science"]}                            │      NULL │ false        │             51 │
│ course      │ is_in      │ {"value": ["Algorithms", "Data Structures", "Calculus I"]} │      NULL │ false        │             38 │
│ age         │ between    │ {"min": 15, "max": 21}                                     │      NULL │ false        │             26 │
└─────────────┴────────────┴──────────────────────────────────